In [1]:
import learn2learn as l2l
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle
import sys
import argparse
import os
from run_MAML_04 import test2 as test_maml


sys.path.insert(1, "..")

from ts_dataset import TSDataset
from base_models import LSTMModel, FCN
from metrics import torch_mae as mae
import copy
from pytorchtools import EarlyStopping
from eval_base_models import test, train, freeze_model
from torch.utils.data import Dataset, DataLoader
from ts_dataset import DomainTSDataset, SimpleDataset
from ts_transform import split_idx_50_50
from pytorchtools import to_torch

In [2]:
meta_info = {"POLLUTION": [5, 50, 14],
             "HR": [32, 50, 13],
             "BATTERY": [20, 50, 3] }




output_directory = "output/"
horizon = 10
output_dim = 1

dataset_name = "BATTERY"
save_model_file = "model.pt"
load_model_file = "model.pt"
lower_trial = 0
upper_trial = 3
learning_rate = 0.01
meta_learning_rate = 0.005
adaptation_steps = 10
batch_size = 20
model_name = "LSTM"
is_test = 1
patience_stopping = 20
epochs = 1000
noise_level = 0.0
noise_type = "additive"

params = {'batch_size': batch_size,
      'shuffle': True,
      'num_workers': 0}

assert model_name in ("FCN", "LSTM"), "Model was not correctly specified"
assert dataset_name in ("POLLUTION", "HR", "BATTERY")

window_size, task_size, input_dim = meta_info[dataset_name]
grid = [0., noise_level]

train_data = pickle.load(  open( "../../Data/TRAIN-"+dataset_name+"-W"+str(window_size)+"-T"+str(task_size)+"-NOML.pickle", "rb" ) )
train_data_ML = pickle.load( open( "../../Data/TRAIN-"+dataset_name+"-W"+str(window_size)+"-T"+str(task_size)+"-ML.pickle", "rb" ) )
validation_data = pickle.load( open( "../../Data/VAL-"+dataset_name+"-W"+str(window_size)+"-T"+str(task_size)+"-NOML.pickle", "rb" ) )
validation_data_ML = pickle.load( open( "../../Data/VAL-"+dataset_name+"-W"+str(window_size)+"-T"+str(task_size)+"-ML.pickle", "rb" ) )
test_data = pickle.load( open( "../../Data/TEST-"+dataset_name+"-W"+str(window_size)+"-T"+str(task_size)+"-NOML.pickle", "rb" ) )
test_data_ML = pickle.load( open( "../../Data/TEST-"+dataset_name+"-W"+str(window_size)+"-T"+str(task_size)+"-ML.pickle", "rb" ) )

In [3]:
save_model_file_ = output_directory+save_model_file
load_model_file_ = output_directory+load_model_file

if is_test == 0:
    test_data = validation_data

train_idx, val_idx, test_idx = split_idx_50_50(test_data.file_idx) if is_test else split_idx_50_50(validation_data.file_idx)
n_domains_in_test = np.max(test_data.file_idx)+1

test_loss_list = []
initial_test_loss_list = []

trial = 0
learning_rate = 0.0001
output_directory = "../../Models/"+dataset_name+"_"+model_name+"_MAML/"+str(trial)+"/"

save_model_file_ = output_directory + "encoder_"+save_model_file
save_model_file_2 = output_directory + save_model_file
load_model_file_ = output_directory + load_model_file

model = LSTMModel( batch_size=batch_size, seq_len = window_size, input_dim = input_dim, n_layers = 2, hidden_dim = 120, output_dim =1)
model2 = nn.Linear(120, 1)

model.cuda()
model2.cuda()

maml = l2l.algorithms.MAML(model2, lr=learning_rate, first_order=False)
model.load_state_dict(torch.load(save_model_file_))
maml.load_state_dict(torch.load(save_model_file_2))

30
[     0  22605  27382  35100  42811  48043  70648  79698 106020 111577
 137899 146928 152677 178460 183947 189407 197725 206043 231826 239879
 247911 253221 278288 303355 308842 333212 338699 344282 352170 360058
 384428]
0
22605
27382
35100
42811
48043
70648
79698
106020
111577
137899
146928
152677
178460
183947
189407
197725
206043
231826
239879
247911
253221
278288
303355
308842
333212
338699
344282
352170
360058


<All keys matched successfully>

In [5]:
learning_rate = 0.0001
n_domains_in_test = np.max(test_data.file_idx)+1
horizon = 0
error_list = []
for domain in range(n_domains_in_test):
    x_test = test_data.x
    y_test = test_data.y


    temp_train_data = SimpleDataset(x=np.concatenate([x_test[ np.concatenate([train_idx[domain],val_idx[domain]])][np.newaxis,:], x_test[ test_idx[domain]][np.newaxis,:]]), 
                                                            y =np.concatenate([ y_test[np.concatenate([train_idx[domain],val_idx[domain]])][np.newaxis,:], y_test[test_idx[domain]][np.newaxis,:]]))


    total_tasks_test = len(test_data_ML)
    

    learner = maml.clone()  # Creates a clone of model
    learner.cuda()
    accum_error = 0.0
    accum_std = 0.0
    count = 0.0
    grid = [0., noise_level]

    input_dim = test_data_ML.x.shape[-1]
    window_size = test_data_ML.x.shape[-2]
    output_dim = test_data_ML.y.shape[-1]

    if is_test:
        step = total_tasks_test//100

    else:
        step = 1

    step = 1 if step == 0 else step

    task = 0


    model2 = nn.Linear(120, 1)
    model2.load_state_dict(copy.deepcopy(maml.module.state_dict()))

    model.cuda()
    model2.cuda()
    #model2.cuda()
    #model2.load_state_dict(copy.deepcopy(maml.module.state_dict()))
    #opt2 = optim.Adam(model2.parameters(), lr=learning_rate)
    #learner = maml.clone() 

    x_spt, y_spt = temp_train_data[task]
    x_qry = temp_train_data.x[(task+1)]
    y_qry = temp_train_data.y[(task+1)]
    #x_qry = test_data_ML.x[(task+1)].reshape(-1, window_size, input_dim)
    #y_qry = test_data_ML.y[(task+1)].reshape(-1, output_dim)

    if model_name == "FCN":
        x_qry = np.transpose(x_qry, [0,2,1])
        x_spt = np.transpose(x_spt, [0,2,1])

    x_spt, y_spt = to_torch(x_spt), to_torch(y_spt)
    x_qry = to_torch(x_qry)
    y_qry = to_torch(y_qry)


    epsilon = grid[np.random.randint(0,len(grid))]

    if noise_type == "additive":
        y_spt = y_spt+epsilon
        y_qry = y_qry+epsilon

    else:
        y_spt = y_spt*(1+epsilon)
        y_qry = y_qry*(1+epsilon)

    opt2 = optim.SGD(list(model2.parameters()), lr=learning_rate)
    #learner.module.train()
    #model2.eval()
    for step in range(adaptation_steps):

        print(step)
        #model2.train()
        for idx in range(x_spt.shape[0]-task_size):

            pred = model2(model.encoder(x_spt[idx:idx+50]))
            error = mae(pred, y_spt[idx:idx+50])
            #print(error)
            opt2.zero_grad()
            error.backward()

            #learner.adapt(error)
            opt2.step()


    #model2.eval()
    #learner.module.eval()
    step = x_qry.shape[0]//255
    for idx in range(0, x_qry.shape[0], step):
        pred = model2(model.encoder(x_qry[idx:idx+step]))
        error = mae(pred, y_qry[idx:idx+step])

        accum_error += error.data
        accum_std += error.data**2
        count += 1

    error = accum_error/count
    
    error_list.append(float(error.cpu().numpy()))
    print(np.mean(error_list))
    print(error_list)

0
1
2
3
4
5
6
7
8
9
0.002504729898646474
[0.002504729898646474]
0
1
2
3
4
5
6
7
8
9
0.0035993530182167888
[0.002504729898646474, 0.004693976137787104]
0
1
2
3
4
5
6
7
8
9
0.0036001050223906836
[0.002504729898646474, 0.004693976137787104, 0.003601609030738473]
0
1
2
3
4
5
6
7
8
9
0.0036755941109731793
[0.002504729898646474, 0.004693976137787104, 0.003601609030738473, 0.003902061376720667]
0
1
2
3
4
5
6
7
8
9
0.00417046369984746
[0.002504729898646474, 0.004693976137787104, 0.003601609030738473, 0.003902061376720667, 0.006149942055344582]
0
1
2
3
4
5
6
7
8
9
0.00386280306459715
[0.002504729898646474, 0.004693976137787104, 0.003601609030738473, 0.003902061376720667, 0.006149942055344582, 0.002324499888345599]
0
1
2
3
4
5
6
7
8
9
0.004012266340266381
[0.002504729898646474, 0.004693976137787104, 0.003601609030738473, 0.003902061376720667, 0.006149942055344582, 0.002324499888345599, 0.004909045994281769]
0
1
2
3
4
5
6
7
8
9
0.004172933782683685
[0.002504729898646474, 0.004693976137787104, 0.0

1
2
3
4
5
6
7
8
9
0.004813728110793118
[0.002504729898646474, 0.004693976137787104, 0.003601609030738473, 0.003902061376720667, 0.006149942055344582, 0.002324499888345599, 0.004909045994281769, 0.0052976058796048164, 0.0060889688320457935, 0.007040795870125294, 0.006661316845566034, 0.0053934394381940365, 0.0034989388659596443, 0.004627198912203312, 0.004729307722300291, 0.0054526543244719505, 0.006637447513639927, 0.006182683166116476, 0.0037476408760994673, 0.0038329085800796747, 0.004533938132226467, 0.0033554094843566418, 0.004903930239379406, 0.005413961596786976, 0.0034088222309947014, 0.006264097988605499]
0
1
2
3
4
5
6
7
8
9
0.004889776680135616
[0.002504729898646474, 0.004693976137787104, 0.003601609030738473, 0.003902061376720667, 0.006149942055344582, 0.002324499888345599, 0.004909045994281769, 0.0052976058796048164, 0.0060889688320457935, 0.007040795870125294, 0.006661316845566034, 0.0053934394381940365, 0.0034989388659596443, 0.004627198912203312, 0.004729307722300291, 0.0

In [9]:
float(error.cpu().numpy())

0.013338057324290276

In [16]:
for idx in range(0, x_qry.shape[0], step):
    pred = model2(model.encoder(x_qry[idx:idx+step]))
    error = mae(pred, y_qry[idx:idx+step])

    accum_error += error.data
    accum_std += error.data**2
    count += 1

error = accum_error/count

In [17]:
error

tensor(0.5524, device='cuda:0')